In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def plt_model(nodes):
    # 提取x, y, z坐标
    x_coords = [node[1] for node in nodes]
    y_coords = [node[2] for node in nodes]
    z_coords = [node[3] for node in nodes]

    # 创建3D图形
    fig = plt.figure(figsize=(20, 6))  # 调整图形尺寸，使其更长
    ax = fig.add_subplot(111, projection='3d')

    # 绘制散点图
    sc = ax.scatter(z_coords, x_coords, y_coords, c='b', marker='.')

    # 设置标签
    ax.set_xlabel('Z Coordinate')  # 将z轴设置为横向显示
    ax.set_ylabel('X Coordinate')
    ax.set_zlabel('Y Coordinate')

    # 设置标题
    ax.set_title('3D Scatter Plot of Node Coordinates (Z-axis Horizontal)')

    # 设置轴的限制
    ax.set_xlim([min(z_coords), max(z_coords)])
    ax.set_ylim([min(x_coords), max(x_coords)])
    ax.set_zlim([min(y_coords), max(y_coords)])

    # 调整图形的长宽比例
    plt.gca().set_box_aspect([10, 1, 2])  # 强制设置坐标轴的比例

    # 调整绘图区域的比例
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

    # 显示图形
    plt.show()


In [9]:
def read_nodes_from_inp(filename, part_name):
    nodes = []
    original_lines = []
    inside_part = False
    with open(filename, 'r') as file:
        for line in file:
            original_lines.append(line)
            line = line.strip()
            if line.startswith('*'):
                if line.startswith('*Part') and f'name={part_name}' in line:
                    inside_part = True
                elif inside_part:
                    if line.startswith('*Node'):
                        continue
                    elif line.startswith('*'):
                        inside_part = False
            elif inside_part and line:
                if ',' in line:
                    node_data = line.split(',')
                    node_id = int(node_data[0].strip())
                    x = float(node_data[1].strip())
                    y = float(node_data[2].strip())
                    z = float(node_data[3].strip())
                    nodes.append((node_id, x, y, z))
    return nodes, original_lines

def write_nodes_to_inp(filename, original_lines, deflected_nodes, part_name):
    deflected_dict = {node[0]: node for node in deflected_nodes}
    inside_part = False
    inside_node = False
    with open(filename, 'w') as file:
        for line in original_lines:
            stripped_line = line.strip()
            if stripped_line.startswith('*'):
                if stripped_line.startswith('*Part') and f'name={part_name}' in stripped_line:
                    inside_part = True
                elif inside_part and stripped_line.startswith('*Node'):
                    inside_node = True
                elif inside_part and stripped_line.startswith('*'):
                    inside_part = False
                    inside_node = False

            if inside_node and ',' in stripped_line:
                node_data = stripped_line.split(',')
                node_id = int(node_data[0].strip())
                if node_id in deflected_dict:
                    x, y, z = deflected_dict[node_id][1:]
                    file.write(f'      {node_id}, {x:.6f}, {y:.6f}, {z:.6f}\n')
                else:
                    file.write(line)
            else:
                file.write(line)

# 定义初期形变函数
def init_defle(A, a, b, x, y):
    defle = a / A * np.sin(np.pi * x / a) * np.sin(np.pi * y / b)
    return defle

# 添加形变到节点数据
def add_deflection(nodes, A, a, b, origin):
    deflected_nodes = []
    for node_id, x, y, z in nodes:
        if origin[1] <= y <= origin[2] and min(origin[3],origin[4]) <= z <= max(origin[3],origin[4]):
            norm_y = abs(y - origin[1])
            norm_z = abs(z - origin[3])
            deflection = init_defle(A, a, b, norm_y, norm_z)
            deflected_nodes.append((node_id, x + deflection, y, z))
        else:
            deflected_nodes.append((node_id, x, y, z))
    return deflected_nodes

def add_def_xp1(nodes, A, a, b, origin):
    deflected_nodes = []
    for node_id, x, y, z in nodes:
        if origin[1] <= x <= origin[2] and min(origin[3],origin[4]) <= z <= max(origin[3],origin[4]):
            norm_y = abs(x - origin[1])
            norm_z = abs(z - origin[3])
            deflection = init_defle(A, a, b, norm_y, norm_z)
            if y > 0:
                deflected_nodes.append((node_id, x, y - deflection, z))
            else:
                deflected_nodes.append((node_id, x, y + deflection, z))
        else:
            deflected_nodes.append((node_id, x, y, z))
    return deflected_nodes

# 读取原始INP文件内容并获取节点数据
inp_filename = '2c2p-C2H-bolt.inp'
part_name = 'C-Ch-Double'
nodes, original_lines = read_nodes_from_inp(inp_filename, part_name)

# 定义不同的板区域和参数
plates = [
    {"name": "plate1", "origin": (0, 10, 390, 1500, 2000), "A": 150, "a": 380, "b": 1000},
    {"name": "plate2", "origin": (0, 10, 390, 1000, 1500), "A": 150, "a": 380, "b": 500},
    {"name": "plate3", "origin": (0, 10, 390, 500, 1000), "A": 150, "a": 380, "b": 500},
    {"name": "plate4", "origin": (0, 10, 390, 500, 0), "A": 150, "a": 380, "b": 1000}
]


flang1 = [
    {"name": "flang1", "origin": (0, 10, 120, 1500, 2000), "A": 100, "a": 220, "b": 1000},
    {"name": "flang2", "origin": (0, 10, 120, 1000, 1500), "A": 100, "a": 220, "b": 500},
    {"name": "flang3", "origin": (0, 10, 120, 500, 1000), "A": 100, "a": 220, "b": 500},
    {"name": "flang4", "origin": (0, 10, 120, 500, 0), "A": 100, "a": 220, "b": 1000}
]

# 对所有板区域进行处理
for plate in plates:
    origin = plate["origin"]
    A = plate["A"]
    a = plate["a"]
    b = plate["b"]
    
    nodes = add_deflection(nodes, A, a, b, origin)

# 对所有法兰区域进行处理
for flang in flang1:
    origin = flang["origin"]
    A = flang["A"]
    a = flang["a"]
    b = flang["b"]
    
    nodes = add_def_xp1(nodes, A, a, b, origin)

# 写入新的整体INP文件
new_inp_filename = inp_filename.replace('.inp', '-deflected.inp')
write_nodes_to_inp(new_inp_filename, original_lines, nodes, part_name)

# 最后可以打印或验证生成的文件
print("Deflected INP file generated.")


Deflected INP file generated.


In [1]:
import apply_init_deflection as aid
import numpy as np

# 读取原始INP文件内容并获取节点数据
inp_filename = '2c2p-C2H-bolt.inp'
part_name = 'C-Ch-Double'

nodes, original_lines = aid.read_nodes_from_inp(inp_filename, part_name)

# 定义不同的板区域和参数
plates = [
    {"name": "plate1", "origin": (0, 10, 390, 1500, 2000), "A": 250, "a": 380, "b": 1000},
    {"name": "plate2", "origin": (0, 10, 390, 1000, 1500), "A": 250, "a": 380, "b": 500},
    {"name": "plate3", "origin": (0, 10, 390, 500, 1000), "A": 250, "a": 380, "b": 500},
    {"name": "plate4", "origin": (0, 10, 390, 500, 0), "A": 250, "a": 380, "b": 1000}
]


# 对所有板区域进行处理
for plate in plates:
    origin = plate["origin"]
    A = plate["A"]
    a = plate["a"]
    b = plate["b"]
    
    nodes = aid.add_deflection(nodes, A, a, b, origin) # 更新节点数据以反映累积的形变

# 写入新的整体INP文件
new_inp_filename = inp_filename.replace('.inp', '-deflected.inp')
aid.write_nodes_to_inp(new_inp_filename, original_lines, nodes, part_name)

# 最后可以打印或验证生成的文件
print("Deflected INP file generated.")


Deflected INP file generated.
